# Geração da planilha de documentos prontos para análise
## Autor: Luiz Felipe Coelho Lemos, Assistente Administrativo
### Autorizado pelo Segundo Secretário Renato Barbosa Ferreira de Andrade, Chefe da Administração
#### **Missão Permanente do Brasil junto às Nações Unidas**

## Passo 1: Instalar Dependências Requeridas

In [ ]:
# instalar módulos requeridos
!pip install pypdf2
# !pip install yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
# instalar corpus da língua portuguesa
!python -m spacy download pt_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 13.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# importar módulos requiridos
import numpy as np # linear algebra
import pandas as pd # data processing, XLSX file output
import nltk # stopwords for text cleaning
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer # TF-IDF calculations for key word extraction
# import yake # YAKE! keyword extractor (may not use b/c bad documentation)
import spacy # NLP capabilities
import string # extra string ops
import re # pattern recognition
import os # read/write capabilities
import PyPDF2 as pypdf # pdf reading

In [ ]:
# incializar objeto da SpaCy usando o corpus português
nlp = spacy.load('pt_core_news_md')

In [ ]:
# baixar embalagem `stopwords` da nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Passo 2: Inicializar Variáveis e Funções

In [ ]:
# incializar estruturas de dádos e outras ferramentas
DOC_SPLITTER_PATTERN = re.compile(r'Página 1/\d+')
DOC_TYPE_PATTERN = re.compile(r'[De|Da|Do]+ ([A-Za-z]+)(?: para ([A-Za-z]+))? em (\d{2}/\d{2}/\d{4})')
DOC_NUM_PATTERN = re.compile(r'Nr[.] (\d{5})')
EXTRA_STOP_WORD_LANGS = ['english', 'french']

data = []
stop_words = set(stopwords.words('portuguese'))

for lang in EXTRA_STOP_WORD_LANGS:
  stop_words = stop_words.union(set(stopwords.words(lang)))

In [ ]:
# definir funções com funcionalidades chaves

'''
Extrair dados do documento
'''
def extract_entry(text: str):
    # Buscar por padrões
    doc_type_match = re.search(DOC_TYPE_PATTERN, text)
    doc_num_match = re.search(DOC_NUM_PATTERN, text)

    # Validação de formato documento, ref. linha 76
    if not (doc_type_match and doc_num_match):
        return None

    # Extrair dados
    msg_type = None
    sender, receiver, date = doc_type_match.group(1, 2, 3)
    year = date[-4:]
    msg_number = doc_num_match.group(1)

    # Determinar se mensagem for TEL, DET, ou CIT
    if sender == 'SERE' and not receiver:
        msg_type = 'CIT'
    elif sender == 'SERE' and receiver:
        msg_type = 'DET'
    else:
        msg_type = 'TEL'

    doc_name = f'{msg_type}_{year}_{msg_number}'

    # Extrair corpo da mensagem
    body_text = None

    if msg_type == 'CIT' or msg_type == 'DET':
        body_text = text[text.index('//'): text.index('EXTERIORES')]
    else:
        body_text = text[text.index('//'): text.index('Sérgio')]

    return doc_name, msg_type, sender, receiver, date, body_text

'''Processo para limpar texto'''
def text_pipeline(s: str):
    s = str.lower(s) # Converter em letras minúsculas
    s = str.translate(s, str.maketrans('','', string.punctuation)) # Remover pontuação
    s = str.join(' ',
                 [word for word in str.split(s) if
                  word not in stop_words]) # Remover stopwords
    return s

## Passo 3: Extrair Dados e Criar Base Inicial

In [ ]:
# ler todos os PDFs para extrair as entradas da planilha
for file in [path for path in os.listdir('./') if path.endswith('.pdf')]:
    pdf_reader = pypdf.PdfReader(f'./{file}')

    total_pages = len(pdf_reader.pages)

    for page_num in range(total_pages):
        text = pdf_reader.pages[page_num].extract_text()

        if DOC_SPLITTER_PATTERN.match(text): # início da mensagem
            for page_num_inner in range(page_num+1, total_pages):
                inner_text = pdf_reader.pages[page_num_inner].extract_text()

                if not DOC_SPLITTER_PATTERN.match(inner_text):
                    text += inner_text
                else:
                    break

            # Validação de formato documento
            extracted_entry = extract_entry(text)

            if not extracted_entry:
                continue # se não for uma página valida, pular para a próxima
            else:
                (documt, tipo, remet, dest, dat, texto) = extract_entry(text)

            data.append(
                {'documento':documt,
                'tipo':tipo,
                'remetente':remet,
                'destinatário':dest,
                'data':dat,
                'texto':texto})

df = pd.DataFrame(data) # criar estrutura DataFrame que se tornará em planilha

## Passo 4: Extrair Outros Dados Relevantes

### I - Palavras Chaves

In [ ]:
doc_series = df['texto'].copy(deep=True) # Criar cópia dos textos pronta para limpeza
text_series = [] # lista de textos depois de passar pelo processamento do spaCy

In [ ]:
# usar spaCy pra excluir palavras em linguas estrangeiras
doc_series = doc_series.apply(nlp)

for doc in doc_series:
  words_list = []
  for token in doc:
    '''
    Adicionar palavra caso tenha uma representação de vetor produzido pelo modelo.
    Ou seja, se a palavra for (supostamente) da lingua portuguêsa, adicioná-la à lista
    '''
    if token.has_vector:
      words_list.append(token.text)
  text_series.append(' '.join(words_list))

In [ ]:
# Limpar textos para processamento
text_series = pd.Series(text_series)
text_series = text_series.apply(text_pipeline)

In [ ]:
# Criar vetores de frequência
cv = CountVectorizer(max_df=0.9, min_df=2)
word_counts = cv.fit_transform(text_series)
features = cv.get_feature_names_out()

In [ ]:
# Aplicar transformador TF-IDF
transformer = TfidfTransformer()
transformer.fit(word_counts)

TfidfTransformer()

In [ ]:
palavras_chaves = [] # inicializar lista de palavras chaves

for entry in text_series:
  count_vector = cv.transform([entry])

  tfidf_vector = transformer.transform(count_vector).tocoo()
  tuples = zip(tfidf_vector.row, tfidf_vector.col, tfidf_vector.data)
  tfidf_vector = sorted(tuples, key=lambda x: x[2], reverse=True) # classificar por relevância

  palavras_chaves.append(features[tfidf_vector[0][1]]) # adicionar palavra mais relevante

df['palavra_chave'] = pd.Series(palavras_chaves)

### II - No. de Occorências de Palavras Chaves Por Texto

In [ ]:
df['no_ocorrencias'] = df.apply(lambda row: str.casefold(row['texto']).count(row['palavra_chave']), axis=1)

,documento,tipo,remetente,destinatário,data,texto,palavra_chave,no_ocorrencias
0,TEL_2024_00001,TEL,DELBRASONU,Exteriores,02/01/2024,//\nAdministração. Comunicações.\nControle de ...,posto,4
1,TEL_2024_00002,TEL,DELBRASONU,Exteriores,02/01/2024,//\nNações Unidas. Desenvolvimento\nsustentáve...,convite,5
2,TEL_2024_00008,TEL,DELBRASONU,Exteriores,03/01/2024,//\nNações Unidas. ONU-Habitat.\nDiretora-exec...,cartas,3
3,TEL_2024_00005,TEL,DELBRASONU,Exteriores,02/01/2024,//\nNações Unidas. CSNU.\nPresidência francesa...,sobre,14
4,TEL_2024_00006,TEL,DELBRASONU,Exteriores,03/01/2024,//\nCooperação Sul-Sul. Fundo\nIBAS. Notas con...,projetos,6
5,TEL_2024_00004,TEL,DELBRASONU,Exteriores,02/01/2024,//\nNações Unidas. CSNU. Mandato\n2022-23. Con...,ucrânia,6
6,TEL_2024_00009,TEL,DELBRASONU,Exteriores,03/01/2024,"//\nNações Unidas. Cooperação\nSul-Sul. ""Perez...",contributions,7
7,TEL_2024_00003,TEL,DELBRASONU,Exteriores,02/01/2024,//\nNações Unidas. Desenvolvimento\nsustentáve...,onu,25
8,TEL_2024_00010,TEL,DELBRASONU,Exteriores,03/01/2024,//\nAdministração. Auxiliares\nlocais. Contrib...,santos,4
9,TEL_2024_00011,TEL,DELBRASONU,Exteriores,03/01/2024,//\nAdministração. Dotação LIM-RF.\nParcela de...,usd,32


## Passo 5: Exportar para Arquivo de Excel

In [ ]:
MODE = input(
'''Favor, selcione um módulo de exibição
c - imprimir no console
e - exportar a uma planilha de excel\n'''
)

EXECUTED = False

while not EXECUTED:
  match MODE:
    case 'c':
      print(f'\n{df}')
      EXECUTED = True
    case 'e':
      df.to_excel('telegramas.xlsx')
      EXECUTED = True
    case _:
      MODE = input('Input inválido\n')

Favor, selcione um módulo de exibição
c - imprimir no console
e - exportar a uma planilha de excel
x
Input inválido
y
Input inválido
c

   a  b
0  1  2
1  2  4
2  3  6
